# the model

## setup

In [1]:
## imports
#  modules
import numpy as np
import pandas as pd
import xarray as xr

import torch
from torch.utils.data import DataLoader

import os

# custom code
import neural_net
import geodataclass
from open_data import open_data_dir
import open_fire_data
from compute_wind_speed import append_wind_speed

import tools

2025-07-31 14:40:49,904 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


In [2]:
## initialise data (takes ~10mins as of 31.07)

DATA_DIR_PATH = '../data'

# open data
data = append_wind_speed(open_data_dir(os.path.join(DATA_DIR_PATH, 'alaska_TEST_DATA')))
prior_data = open_data_dir(os.path.join(DATA_DIR_PATH, 'alaska_prior'))
fire_data = pd.read_csv(os.path.join(DATA_DIR_PATH, 'FIRE/alaska_range_csv/data.csv'))

# create gdc
input_data = geodataclass.FlattenedData(data, prior_data)
fire_data = open_fire_data.FlattenedTruthTable(fire_data)

ds = neural_net.geoDataset(input_data, fire_data)
data_loader = DataLoader(ds)

reading ../data\alaska_TEST_DATA\2010-01_input_data


skipping variable: paramId==235 shortName='skt'
Traceback (most recent call last):
  File "c:\Users\nookh\miniconda3\envs\ml_ff\Lib\site-packages\cfgrib\dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
    ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nookh\miniconda3\envs\ml_ff\Lib\site-packages\cfgrib\dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
    ...<2 lines>...
    )
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1262217600, 1262304000, 1262390400, 1262476800, 1262563200,
       1262649600, 1262736000, 1262822400, 1262908800, 1262995200,
       1263081600, 1263168000, 1263254400, 1263340800, 1263427200,
       1263513600, 1263600000, 1263686400, 1263772800, 1263859200,
       1263945600, 1264032000, 1264118400, 1264204800, 1264291200,
       1264377600, 1264464000, 1264550400, 1264636800, 1264723200,
       1264809600, 1264896000

[read_grib_file] run time: 8.123658657073975s
reading ../data\alaska_TEST_DATA\2010-02_input_data


skipping variable: paramId==235 shortName='skt'
Traceback (most recent call last):
  File "c:\Users\nookh\miniconda3\envs\ml_ff\Lib\site-packages\cfgrib\dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
    ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nookh\miniconda3\envs\ml_ff\Lib\site-packages\cfgrib\dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
    ...<2 lines>...
    )
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1264896000, 1264982400, 1265068800, 1265155200, 1265241600,
       1265328000, 1265414400, 1265500800, 1265587200, 1265673600,
       1265760000, 1265846400, 1265932800, 1266019200, 1266105600,
       1266192000, 1266278400, 1266364800, 1266451200, 1266537600,
       1266624000, 1266710400, 1266796800, 1266883200, 1266969600,
       1267056000, 1267142400, 1267228800, 1267315200])) new_value=Variable(dimensions=('time',)

[read_grib_file] run time: 7.452829122543335s
done!
[open_data_dir] run time: 15.577849626541138s
reading ../data\alaska_prior\2009-06_proxy_data
[read_grib_file] run time: 1.7808737754821777s
reading ../data\alaska_prior\2009-07_proxy_data
[read_grib_file] run time: 1.8458061218261719s
reading ../data\alaska_prior\2009-08_proxy_data
[read_grib_file] run time: 1.9039430618286133s
reading ../data\alaska_prior\2009-09_proxy_data
[read_grib_file] run time: 1.8139333724975586s
reading ../data\alaska_prior\2009-10_proxy_data
[read_grib_file] run time: 1.5833532810211182s
reading ../data\alaska_prior\2009-11_proxy_data
[read_grib_file] run time: 1.5775065422058105s
reading ../data\alaska_prior\2009-12_proxy_data
[read_grib_file] run time: 1.6034786701202393s
done!
[open_data_dir] run time: 12.111928224563599s
[<class 'geodataclass.FlattenedData'>] begin init
computing proxes: var_names = ('tp', 't2m')
No data for month = '2009-06' // e = KeyError("not all values found in index 'time'. Try se

In [3]:
## set hyperparameters / other constants
len(input_data)

6929376

In [9]:
input_data[200000, 7]

<xarray.DataArray 'tot_tp' ()> Size: 4B
array(nan, dtype=float32)
Coordinates:
    time        datetime64[ns] 8B 2010-01-02
    step        float64 8B 17.0
    latitude    float64 8B 64.4
    longitude   float64 8B -151.4
    number      int64 8B 0
    surface     float64 8B 0.0
    valid_time  datetime64[ns] 8B ...

In [ ]:
## initialise model

nn = neural_net.DNN()

## training


In [ ]:
## train model

In [ ]:
## test model / generate visualisations